In [2]:
from __future__ import print_function 
import numpy as np
import tensorflow as tf 

import os
import sys

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Lambda
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.layers import SimpleRNN, GRU, LSTM, Bidirectional

import keras.backend as K
from keras import optimizers

Using TensorFlow backend.


## Load the dataset

In [3]:
f = np.load('data_and_embedding.npz')

In [4]:
num_words = f['num_words']
embedding_dim = f['embedding_dim']
max_sequence_length = f['max_sequence_length']

data = f['data']
labels = f['labels']

embedding_matrix = f['embedding_matrix']

In [5]:
validation_split = 0.2 
epoch = 10

In [6]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(validation_split * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

In [8]:
print(x_train.shape)
print(y_train.shape)

print(type(x_train))
print(type(y_train))

(311902, 1000)
(311902, 6)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


## Simple RNN

### Build the model 

In [7]:
embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=max_sequence_length,
                            trainable=False)

In [8]:
def embedding_mean(x):
    return tf.reduce_mean(x, axis=1)

In [18]:
sequence_input = Input(shape=(max_sequence_length, ), dtype='int32')
# print(sequence_input.shape)
embedded_sequences = embedding_layer(sequence_input)
# print(embedded_sequences.shape)
x = SimpleRNN(50, dropout=0.2, recurrent_dropout=0.2)(embedded_sequences)
print(x.shape)
# x = Lambda(embedding_mean)(embedded_sequences)
# print(x.shape)
preds = Dense(6, activation='softmax')(x)
# print(preds.shape)

model_rnn_final_state = Model(sequence_input, preds)

(?, 50)


In [ ]:
model_rnn_final_state.summary()

### Train the model

In [ ]:
adam = optimizers.Adam(clipnorm=1.)

model_rnn_final_state.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['acc'])

In [ ]:
import time 
start_time = time.time()

model_rnn_final_state.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val))

print("Training time: ", time.time() - start_time)

### Save the model

In [ ]:
model_rnn_final_state.save('models/RNN.h5')

## LSTM

### Build the model 

In [7]:
embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=max_sequence_length,
                            trainable=False)

In [8]:
def embedding_mean(x):
    return tf.reduce_mean(x, axis=1)

In [16]:
sequence_input = Input(shape=(max_sequence_length, ), dtype='int32')
print(sequence_input.shape)
embedded_sequences = embedding_layer(sequence_input)
print(embedded_sequences.shape)
x = LSTM(50, dropout=0.2, recurrent_dropout=0.2)(embedded_sequences)
print(x.shape)
# x = Lambda(embedding_mean)(x)
# print(x.shape)
preds = Dense(6, activation='softmax')(x)
print(preds.shape)

model_lstm_final_state = Model(sequence_input, preds)

(?, 1000)
(?, 1000, 100)
(?, 50)
(?, 6)


In [14]:
model_lstm_final_state.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, array(1000))       0         
_________________________________________________________________
embedding_1 (Embedding)      (None, array(1000), array 2000000   
_________________________________________________________________
lstm_4 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 306       
Total params: 2,030,506.0
Trainable params: 30,506.0
Non-trainable params: 2,000,000.0
_________________________________________________________________


### Train the model

In [15]:
model_lstm_final_state.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [ ]:
import time 
start_time = time.time()

model_lstm_final_state.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val))

print("Training time: ", time.time() - start_time)

### Save the model

In [ ]:
model_lstm_final_state.save('models/LSTM.h5')

## GRU

### Build the model 

In [7]:
embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=max_sequence_length,
                            trainable=False)

In [8]:
def embedding_mean(x):
    return tf.reduce_mean(x, axis=1)

In [17]:
sequence_input = Input(shape=(max_sequence_length, ), dtype='int32')
# print(sequence_input.shape)
embedded_sequences = embedding_layer(sequence_input)
print(embedded_sequences.shape)
x = GRU(50, dropout=0.2, recurrent_dropout=0.2)(embedded_sequences)
print(x.shape)
# x = Lambda(embedding_mean)(embedded_sequences)
# print(x.shape)
preds = Dense(6, activation='softmax')(x)
# print(preds.shape)

model_gru_final_state = Model(sequence_input, preds)

(?, 1000, 100)
(?, 50)


In [ ]:
model_gru_final_state.summary()

### Train the model

In [ ]:
model_gru_final_state.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [ ]:
import time 
start_time = time.time()

model_gru_final_state.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val))

print("Training time: ", time.time() - start_time)

### Save the model

In [ ]:
model_gru_final_state.save('models/GRU.h5')

## Bidirectional LSTM

### Build the model 

In [7]:
embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=max_sequence_length,
                            trainable=False)

In [8]:
def embedding_mean(x):
    return tf.reduce_mean(x, axis=1)

In [20]:
sequence_input = Input(shape=(max_sequence_length, ), dtype='int32')
# print(sequence_input.shape)
embedded_sequences = embedding_layer(sequence_input)
# print(embedded_sequences.shape)
x = Bidirectional(LSTM(50, dropout=0.2, recurrent_dropout=0.2))(embedded_sequences)
print(x.shape)
# x = Lambda(embedding_mean)(embedded_sequences)
# print(x.shape)
preds = Dense(6, activation='softmax')(x)
# print(preds.shape)

model_bidirlstm_final_state = Model(sequence_input, preds)

(?, 100)


In [21]:
model_bidirlstm_final_state.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, array(1000))       0         
_________________________________________________________________
embedding_1 (Embedding)      (None, array(1000), array 2000000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100)               60400     
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 606       
Total params: 2,061,006.0
Trainable params: 61,006.0
Non-trainable params: 2,000,000.0
_________________________________________________________________


### Train the model

In [ ]:
model_bidirlstm_final_state.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [ ]:
import time 
start_time = time.time()

model_bidirlstm_final_state.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val))

print("Training time: ", time.time() - start_time)

### Save the model

In [ ]:
model_bidirlstm_final_state.save('models/BidirectionalLSTM.h5')